Image Labeling GUI (Kanji/Katakana/Hiragana/Other)

This notebook allows you to label images.

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import matplotlib.pyplot as plt

# Load existing labels if available
labels = {}

label_file = "labels.csv"
if os.path.exists(label_file):
    try:
        df = pd.read_csv(label_file)
        for filepath, label in zip(df['filepath'], df['label']):
            labels[filepath] = label
        print(f"Loaded {len(labels)} existing labels.")
    except Exception as e:
        print(f"Error loading existing labels: {e}")
else:
    print("No existing labels found. Starting fresh.")

# Load all image paths
image_folder = 'segmented_chars_img/RC04844/page_1/'  # <-- Modify to your folder
image_paths = sorted([os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png')])

# Initialize widgets
out = widgets.Output()
prev_button = widgets.Button(description='Previous', button_style='info')
kanji_button = widgets.Button(description='Kanji', button_style='danger')
katakana_button = widgets.Button(description='Katakana', button_style='success')
hiragana_button = widgets.Button(description='Hiragana', button_style='warning')
other_button = widgets.Button(description='Other')

jump_dropdown = widgets.Dropdown(
    options=[(os.path.basename(p), idx) for idx, p in enumerate(image_paths)],
    description='Jump to:',
    layout=widgets.Layout(width='300px')
)

current_index = 0
current_label = None

# Show image
def show_image(index):
    with out:
        clear_output(wait=True)
        if 0 <= index < len(image_paths):
            img = Image.open(image_paths[index])
            plt.figure(figsize=(2.5, 2.5))
            plt.imshow(img, cmap='gray')
            plt.axis('off')
            plt.show()
            print(f"Now labeling: {os.path.basename(image_paths[index])}")
            rel_path = os.path.relpath(image_paths[index], start='.')
            if rel_path in labels:
                print(f"Existing label: {labels[rel_path]}")
            else:
                print("No label yet.")

# Save label
def save_current_label():
    rel_path = os.path.relpath(image_paths[current_index], start='.')
    if current_label is not None:
        labels[rel_path] = current_label

    backup_path = "labels_backup.csv"
    if os.path.exists(label_file):
        if os.path.exists(backup_path):
            os.remove(backup_path)
        os.rename(label_file, backup_path)
        print(f"Backup created: {backup_path}")

    rel_paths = list(labels.keys())
    df = pd.DataFrame({
        'filepath': rel_paths,
        'label': [labels[p] for p in rel_paths]
    })
    df.to_csv(label_file, index=False)
    print(f"Saved labels to {label_file}.")

#  Move to next
def move_next():
    global current_index
    if current_index < len(image_paths) - 1:
        current_index += 1
    show_image(current_index)

# Move to previous
def on_prev_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
    show_image(current_index)

# Label buttons
def label_and_next(label_type):
    global current_label
    current_label = label_type
    save_current_label()
    move_next()

def on_kanji_clicked(b):
    label_and_next('Kanji')

def on_katakana_clicked(b):
    label_and_next('Katakana')

def on_hiragana_clicked(b):
    label_and_next('Hiragana')

def on_other_clicked(b):
    label_and_next('Other')

# Jump dropdown handler
def on_jump_selected(change):
    global current_index
    if change['type'] == 'change' and change['name'] == 'value':
        current_index = change['new']
        show_image(current_index)

jump_dropdown.observe(on_jump_selected)

# Connect buttons
prev_button.on_click(on_prev_clicked)
kanji_button.on_click(on_kanji_clicked)
katakana_button.on_click(on_katakana_clicked)
hiragana_button.on_click(on_hiragana_clicked)
other_button.on_click(on_other_clicked)

# Display GUI
display(out, 
        widgets.HBox([prev_button, jump_dropdown]),
        widgets.HBox([kanji_button, katakana_button, hiragana_button, other_button]))

show_image(current_index)

